# Bases de Datos con SQLalchemy

## Simple Database with SQLalchemy

- Instalacion de SQLalchemy
  ```bash
  pip install sqlalchemy
  ```
___
  
- Importaciones:
  - create_engine: Función de SQLAlchemy que se utiliza para establecer una conexión con la base de datos.
  - text: Función de SQLAlchemy que permite crear consultas SQL a partir de cadenas de texto.
___

In [1]:
import sqlalchemy

In [3]:
sqlalchemy.__version__

'2.0.38'

In [4]:
from sqlalchemy import create_engine, text

- Creación del motor de base de datos:

  - create_engine('sqlite:///ecommerce.db', echo=True): Crea un motor de base de datos para interactuar con un archivo SQLite llamado ecommerce.db.
  - para usar una base en memoria ram usar create_engine('sqlite:///:memory:')
  - El parámetro echo=True es útil para depurar, ya que muestra las consultas SQL que se ejecutan.

In [5]:
engine = create_engine('sqlite:///:memory:', echo=True)

- Conexión y ejecución de la consulta:
  - with engine.connect() as connection: Abre una conexión al motor de la base de datos. El uso de with asegura que la conexión se cierre automáticamente cuando se sale del bloque.
  - connection.execute(text("SELECT 1")): Ejecuta una consulta SQL que simplemente selecciona el número 1. Esto es una prueba básica para verificar que la conexión a la base de datos funciona correctamente.
- Obtención y muestra del resultado:
  - result.scalar(): Devuelve el valor escalar (único) del resultado de la consulta. En este caso, debería ser 1.
  - print(result.scalar()): Imprime el resultado de la consulta, que debería ser 1 si la conexión a la base de datos es exitosa.

In [6]:
with engine.connect() as connection:
    result = connection.execute(text("SELECT 1"))
    # Fetch and print the result
    print(result.scalar()) # This should print 1 if the connection is successful

2025-05-19 19:50:09,870 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-19 19:50:09,881 INFO sqlalchemy.engine.Engine SELECT 1
2025-05-19 19:50:09,883 INFO sqlalchemy.engine.Engine [generated in 0.01362s] ()
1
2025-05-19 19:50:09,886 INFO sqlalchemy.engine.Engine ROLLBACK


> Este código es una manera simple de verificar que tu configuración de SQLAlchemy y la conexión a la base de datos están funcionando correctamente.

## Implementacion de un Esquema

Tengo una base de datos SQLite configurada en Python con SQLAlchemy. ¿Podrían ayudarme a implementar tablas para el siguiente esquema?

. Users:
   - id (Primary Key)
   - name
   - email (Unique)

2. Products:
   - id (Primary Key)
   - name
   - price

3. Orders:
   - id (Primary Key)
   - user_id (Foreign Key referencing Users)

4. Order_items:
   - id (Primary Key)
   - order_id (Foreign Key referencing Orders)
   - product_id (Foreign Key referencing Products)
   - quantity

Usa sqlalchemy pero normal, no ORM


ChatGPT dijo:
Claro, aquí tienes cómo implementar las tablas que describes usando SQLAlchemy con estilo SQL Expression Language (no ORM). Este enfoque es más explícito y cercano al SQL puro.

### ✅ Modo Imperativo (Core)

- Define tablas con Table(...), columnas, claves foráneas, etc.
- No usa clases ni objetos.
- Es como escribir SQL estructurado en Python.
- No es ORM.

### ✅ Modo Declarativo + ORM 

- Definelas tablas como class Usuario(Base)
- Usa POO: defines clases que representan tablas.
- Estas clases permiten crear instancias (objetos) que representan filas.
- Te permite usar session.add(), session.query(), etc.

Es el modo ORM de SQLAlchemy, usando el sistema declarativo para definir los modelos.

Importaciones

In [8]:
# Estructura de la base de datos
from sqlalchemy import create_engine, MetaData, Table, Column

# Tipos de datos
from sqlalchemy import Integer, String, Float

# Relaciones y restricciones
from sqlalchemy import ForeignKey, UniqueConstraint

## Creacion de la base de Datos (Modo Imperativo)

In [9]:
# Crear motor y metadatos
engine = create_engine('sqlite:///:memory:', echo=True)
metadata = MetaData()

Tabla de usuarios

In [10]:
# Tabla Users
users = Table(
    'users', metadata,
    Column('id', Integer, primary_key = True),
    Column('name', String, nullable = False),
    Column('email', String, nullable = False, unique = True)
)

- Usuarios con ID único, nombre y email.

- email es único (unique=True), no puede repetirse.

In [11]:
# Tabla Products
products = Table(
    'products', metadata,
    Column('id', Integer, primary_key=True),
    Column('name', String, nullable=False),
    Column('price', Float, nullable=False)
)

- Productos con ID, nombre y precio.

In [13]:
# Tabla Orders
orders = Table(
    'orders', metadata,
    Column('id', Integer, primary_key=True),
    Column('user_id', Integer, ForeignKey('users.id'), nullable = False)
)

InvalidRequestError: Table 'orders' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

- Pedidos realizados por usuarios.

  - user_id es clave foránea que apunta al ID de la tabla users.

In [14]:
# Tabla Order_items
order_items = Table(
    'order_items', metadata,
    Column('id', Integer, primary_key=True),
    Column('order_id', Integer, ForeignKey('orders.id'), nullable=False),
    Column('product_id', Integer, ForeignKey('products.id'), nullable=False),
    Column('quantity', Integer, nullable=False)
)

-- Detalle de productos dentro de un pedido.
  - Cada fila representa un producto específico en un pedido dado.
  - Tiene claves foráneas a orders.id y products.id.

In [15]:
# Crear las tablas en la base de datos
metadata.create_all(engine)

2025-05-19 20:02:32,854 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-19 20:02:32,856 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2025-05-19 20:02:32,858 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-19 20:02:32,860 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2025-05-19 20:02:32,861 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-19 20:02:32,863 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("products")
2025-05-19 20:02:32,869 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-19 20:02:32,871 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("products")
2025-05-19 20:02:32,873 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-19 20:02:32,875 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("orders")
2025-05-19 20:02:32,879 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-19 20:02:32,881 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("orders")
2025-05-19 20:02:32,883 INFO sqlalchemy.engine.Engine [raw sql] ()
202

- Se crean todas las tablas definidas en el objeto metadata en la base de datos conectada por engine.



### En SQLAlchemy, UniqueConstraint se usa para asegurar que una o más columnas tengan valores únicos en combinación, a nivel de tabla.

- ¿Por qué usar UniqueConstraint?

Cuando quieres garantizar que una columna o una combinación de columnas no se repita.

A diferencia de unique=True (que se aplica a una sola columna), UniqueConstraint puede aplicarse a varias columnas al mismo tiempo.

- Ejemplo simple (una columna):

En la tabla users, cuando escribes:

```python
Column('email', String, unique=True)
```

Esto es equivalente a:

```python
UniqueConstraint('email', name='uix_email')
```

uix_email es simplemente el nombre que tú le das a la restricción de unicidad. Sirve para:

- Identificar la restricción en la base de datos.
- Poder referenciarla si más adelante quieres modificarla, eliminarla o capturar errores relacionados.
- Darle un nombre legible y significativo.

> ¿Por qué usar un nombre?

> Cuando no le das un nombre, SQLAlchemy (o el motor SQL) genera uno automático, que puede ser poco claro, como sqlite_autoindex_users_1.
> Dándole un nombre como uix_email, el significado es claro:

- uix → convención común para "unique index" o "unique constraint"
- email → columna afectada

### Ejemplo con múltiples columnas:

Supón que tienes una tabla de asistencia donde un usuario no puede registrar asistencia más de una vez por día:

```python
from sqlalchemy import Table, Column, Integer, Date, ForeignKey, UniqueConstraint

attendance = Table(
    'attendance', metadata,
    Column('id', Integer, primary_key=True),
    Column('user_id', Integer, ForeignKey('users.id')),
    Column('date', Date),
    UniqueConstraint('user_id', 'date', name='uix_user_date')
)
```

Esto significa que no puede haber dos filas con el mismo user_id y date. Es útil para reglas de negocio más complejas.

¿Cuándo usar unique=True vs UniqueConstraint?

| Uso                                           | Mejor opción            |
| --------------------------------------------- | ----------------------- |
| Una sola columna debe ser única               | `unique=True`           |
| Combinación de varias columnas debe ser única | `UniqueConstraint(...)` |


## Consideraciones de Seguridad

## ¿Cuáles son las cosas a tener en cuenta al implementar la operación de creación de una base de datos?


ChatGPT dijo:
Buena pregunta. Al implementar la creación de una base de datos (como cuando usás SQLAlchemy con metadata.create_all(engine)), hay varios aspectos clave que deberías tener en cuenta para evitar errores y asegurarte de que la base sea robusta y útil desde el inicio.

✅ 1. Definir bien el esquema desde el principio
Asegurate de que cada tabla tenga una clave primaria (primary_key).

Usá tipos de datos apropiados (String, Integer, Float, etc.).

Definí bien las relaciones con ForeignKey si hay referencias entre tablas.

✅ 2. Restricciones
unique=True o UniqueConstraint() para evitar duplicados.

nullable=False en columnas obligatorias.

CheckConstraint si necesitás validar rangos (por ejemplo, price > 0).

✅ 3. Nombre de las tablas y columnas
Usá nombres consistentes, en minúsculas y con guiones bajos (snake_case).

Evitá palabras reservadas de SQL como order, select, etc.

✅ 4. Manejo de errores
Asegurate de que si la base ya existe, no se sobreescriba sin querer.

Capturá errores si usás create_all() varias veces para evitar problemas con restricciones duplicadas.

✅ 5. Persistencia y motor
Asegurate de que create_engine() apunte al archivo o base correcta (SQLite, PostgreSQL, etc.).

Para SQLite: asegurate de que el archivo tenga permisos de escritura si se va a modificar.

✅ 6. Versión de la base de datos y migraciones
SQLAlchemy no borra ni modifica automáticamente tablas existentes. Si el esquema cambia, vas a necesitar una herramienta como Alembic para hacer migraciones controladas.

_
✅ 7. Pruebas iniciales
Después de crear las tablas, es buena práctica probar inserciones básicas para verificar que las restricciones funcionen como esperás.

✅ 8. Documentación del esquema
Mantené una descripción clara de qué representa cada tabla y campo.

Esto ayuda a otros desarrolladores o a vos mismo en el futuro.

## 🧩 Checklist en Código – SQLAlchemy + SQLite (Modo Imperativo)

In [16]:
from sqlalchemy import (
    create_engine, MetaData, Table, Column,
    Integer, String, Float, ForeignKey, UniqueConstraint, inspect
)
import os

In [17]:
# 1. Crear motor: asegurate de no sobrescribir accidentalmente
db_path = 'prueba.db'
if os.path.exists(db_path):
    print(f"⚠️ Ya existe la base de datos: {db_path}")
else:
    print(f"✅ Se va a crear la base: {db_path}")

✅ Se va a crear la base: prueba.db


In [18]:
engine = create_engine(f'sqlite:///{db_path}', echo=True )
metadata = MetaData()

In [19]:
# 2. Definir esquema: con claves primarias y restricciones
users = Table(
    'users', metadata,
    Column('id', Integer, primary_key=True),
    Column('name', String, nullable=False),
    Column('email', String, nullable=False),
    UniqueConstraint('email', name='uix_email')
)

products = Table(
    'products', metadata,
    Column('id', Integer, primary_key=True),
    Column('name', String, nullable=False),
    Column('price', Float, nullable=False)
)

orders = Table(
    'orders', metadata,
    Column('id', Integer, primary_key=True),
    Column('user_id', Integer, ForeignKey('users.id'), nullable=False)
)

order_items = Table(
    'order_items', metadata,
    Column('id', Integer, primary_key=True),
    Column('order_id', Integer, ForeignKey('orders.id'), nullable=False),
    Column('product_id', Integer, ForeignKey('products.id'), nullable=False),
    Column('quantity', Integer, nullable=False)
)

In [20]:
# 3. Crear tablas (si no existen)
metadata.create_all(engine)

2025-05-19 20:14:34,906 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-19 20:14:34,909 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2025-05-19 20:14:34,911 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-19 20:14:34,915 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2025-05-19 20:14:34,916 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-19 20:14:34,920 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("products")
2025-05-19 20:14:34,922 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-19 20:14:34,925 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("products")
2025-05-19 20:14:34,927 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-19 20:14:34,929 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("orders")
2025-05-19 20:14:34,933 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-19 20:14:34,936 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("orders")
2025-05-19 20:14:34,938 INFO sqlalchemy.engine.Engine [raw sql] ()
202

In [21]:
# 4. Verificar las tablas creadas (inspector)
inspector = inspect(engine)
created_tables = inspector.get_table_names()
print("\n📋 Tablas creadas:")
for t in created_tables:
    print(f" - {t}")

2025-05-19 20:15:28,864 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-19 20:15:28,868 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-05-19 20:15:28,873 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-19 20:15:28,885 INFO sqlalchemy.engine.Engine ROLLBACK

📋 Tablas creadas:
 - order_items
 - orders
 - products
 - users


In [23]:
# 5. Verificar claves primarias y restricciones únicas
print("\n🔍 Revisión de restricciones:")
for table_name in created_tables:
    pk = inspector.get_pk_constraint(table_name)
    uniques = inspector.get_unique_constraints(table_name)
    print(f"→ {table_name}:")
    print(f"   - Primary Key: {pk['constrained_columns']}")
    if uniques:
        for uc in uniques:
            print(f"   - Unique: {uc['column_names']} (nombre: {uc['name']})")
    else:
        print("   - Sin restricciones únicas")


🔍 Revisión de restricciones:
→ order_items:
   - Primary Key: ['id']
   - Sin restricciones únicas
→ orders:
   - Primary Key: ['id']
   - Sin restricciones únicas
→ products:
   - Primary Key: ['id']
   - Sin restricciones únicas
→ users:
   - Primary Key: ['id']
   - Unique: ['email'] (nombre: uix_email)


#### ✅ Este script hace:

- Verifica si el archivo de base existe antes de crear.
- Define el esquema completo con claves primarias y restricciones.
- Crea las tablas (sin duplicar si ya existen).
- Usa inspect() para revisar qué se creó realmente.
- Imprime claves primarias y restricciones únicas por tabla.

### Creacion de un Nuevo Usuario en modo declarativo

In [24]:
from sqlalchemy import insert
from sqlalchemy.exc import IntegrityError

In [25]:
def crear_usuario(nombre: str, email: str) -> None:
    """
    Inserta un nuevo usuario en la tabla `users`.
    Lanza una excepción si el email ya existe o hay otro problema.
    """
    stmt = insert(users).values(name=nombre, email=email)

    # `engine.begin()` abre una transacción y hace commit automático si todo va bien
    try:
        with engine.begin() as conn:
            conn.execute(stmt)
        print(f"✅ Usuario '{nombre}' agregado.")
    except IntegrityError:
        print("Ese email ya está registrado.")
    except Exception as exc:
        print(f"❌ No se pudo agregar el usuario: {exc}")

In [26]:
# Ejemplo de uso
crear_usuario("María López", "maria@example.com")

2025-05-19 20:22:43,756 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-19 20:22:43,768 INFO sqlalchemy.engine.Engine INSERT INTO users (name, email) VALUES (?, ?)
2025-05-19 20:22:43,769 INFO sqlalchemy.engine.Engine [generated in 0.00198s] ('María López', 'maria@example.com')
2025-05-19 20:22:43,773 INFO sqlalchemy.engine.Engine COMMIT
✅ Usuario 'María López' agregado.


In [27]:
crear_usuario("Pedro Gómez", "pedro@example.com")

2025-05-19 20:23:20,880 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-19 20:23:20,883 INFO sqlalchemy.engine.Engine INSERT INTO users (name, email) VALUES (?, ?)
2025-05-19 20:23:20,892 INFO sqlalchemy.engine.Engine [cached since 37.13s ago] ('Pedro Gómez', 'pedro@example.com')
2025-05-19 20:23:20,899 INFO sqlalchemy.engine.Engine COMMIT
✅ Usuario 'Pedro Gómez' agregado.


In [28]:
# Duplicado → disparará IntegrityError por la restricción única
crear_usuario("Alguien", "maria@example.com")

2025-05-19 20:23:38,464 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-19 20:23:38,469 INFO sqlalchemy.engine.Engine INSERT INTO users (name, email) VALUES (?, ?)
2025-05-19 20:23:38,472 INFO sqlalchemy.engine.Engine [cached since 54.7s ago] ('Alguien', 'maria@example.com')
2025-05-19 20:23:38,478 INFO sqlalchemy.engine.Engine ROLLBACK
Ese email ya está registrado.


In [29]:
crear_usuario("emi","noes_unmail")

2025-05-19 20:24:38,595 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-19 20:24:38,598 INFO sqlalchemy.engine.Engine INSERT INTO users (name, email) VALUES (?, ?)
2025-05-19 20:24:38,600 INFO sqlalchemy.engine.Engine [cached since 114.8s ago] ('emi', 'noes_unmail')
2025-05-19 20:24:38,606 INFO sqlalchemy.engine.Engine COMMIT
✅ Usuario 'emi' agregado.


Funcion con Validacion de email

In [30]:
import re
from sqlalchemy import insert
from sqlalchemy.exc import IntegrityError

EMAIL_REGEX = r'^[\w\.-]+@[\w\.-]+\.\w+$'

def crear_usuario(nombre: str, email: str):
    def es_email_valido(email: str) -> bool:
        return re.match(EMAIL_REGEX, email) is not None

    if not es_email_valido(email):
        # print(f"❌ Email inválido: '{email}'")
        raise ValueError (f"❌ Email inválido: '{email}'")

    stmt = insert(users).values(name=nombre, email=email)

    try:
        with engine.begin() as conn:
            conn.execute(stmt)
        print(f"✅ Usuario '{nombre}' agregado.")
    except IntegrityError:
        print("❌ Ese email ya está registrado.")
    except Exception as exc:
        print(f"❌ Error inesperado: {exc}")

In [31]:
crear_usuario("emi","noes_unmail")

ValueError: ❌ Email inválido: 'noes_unmail'

- Opción 2: Usar una librería como email-validator (más robusta)
```bash
pip install email-validator
```

Y luego:

```python
from email_validator import validate_email, EmailNotValidError

def es_email_valido(email: str) -> bool:
    try:
        validate_email(email)
        return True
    except EmailNotValidError:
        return False
```
Esto te protege mejor contra emails con formatos incorrectos, dominios inválidos, etc.

Resumen
Usa regex si solo te interesa una validación básica (más rápida y sin dependencias).

Usa email-validator si necesitas una validación más completa y robusta.

¿Quieres que también rechace correos con dominios específicos (como temporales) o que sea sensible a mayúsculas, etc.? Puedo ayudarte a extender la validación si lo necesitas.

Funcion para leer todos los usuarios

In [32]:
from sqlalchemy import select

In [35]:
# Función para leer todos los usuarios
def leer_todos_los_usuarios(table=False):
    with engine.connect() as conn:
        stmt = select(users)
        result = conn.execute(stmt)
        if table:
            return [dict(row) for row in result.mappings()] # devuelve la columna
        return [row for row in result] # solo los datos

In [36]:
# Ejemplo de uso
usuarios = leer_todos_los_usuarios()
for usuario in usuarios:
    print(usuario)

2025-05-19 20:31:41,608 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-19 20:31:41,613 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.email 
FROM users
2025-05-19 20:31:41,615 INFO sqlalchemy.engine.Engine [cached since 76.71s ago] ()
2025-05-19 20:31:41,623 INFO sqlalchemy.engine.Engine ROLLBACK
(1, 'María López', 'maria@example.com')
(2, 'Pedro Gómez', 'pedro@example.com')
(3, 'emi', 'noes_unmail')


✅ Funcion para eliminar un usuario por cualquier campo

In [37]:
usuarios = leer_todos_los_usuarios(True)
for usuario in usuarios:
    print(usuario)

2025-05-19 20:32:26,429 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-19 20:32:26,435 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.email 
FROM users
2025-05-19 20:32:26,442 INFO sqlalchemy.engine.Engine [cached since 121.5s ago] ()
2025-05-19 20:32:26,445 INFO sqlalchemy.engine.Engine ROLLBACK
{'id': 1, 'name': 'María López', 'email': 'maria@example.com'}
{'id': 2, 'name': 'Pedro Gómez', 'email': 'pedro@example.com'}
{'id': 3, 'name': 'emi', 'email': 'noes_unmail'}


In [38]:
from sqlalchemy import delete

In [39]:
def eliminar_usuario_por(**filtros):
    """
    Elimina un usuario por uno o más filtros: id=1, name='Juan', email='a@b.com'
    Retorna la cantidad de filas eliminadas.
    """
    with engine.connect() as conn:
        condiciones = [getattr(users.c, campo) == valor for campo, valor in filtros.items()]
        stmt = delete(users).where(*condiciones)
        result = conn.execute(stmt)
        conn.commit()
        return result.rowcount

In [40]:
eliminar_usuario_por(name="emi")

2025-05-19 20:36:32,792 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-19 20:36:32,800 INFO sqlalchemy.engine.Engine DELETE FROM users WHERE users.name = ?
2025-05-19 20:36:32,813 INFO sqlalchemy.engine.Engine [generated in 0.02086s] ('emi',)
2025-05-19 20:36:32,831 INFO sqlalchemy.engine.Engine COMMIT


1

In [41]:
usuarios = leer_todos_los_usuarios()
for usuario in usuarios:
    print(usuario)

2025-05-19 20:36:56,586 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-19 20:36:56,589 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.email 
FROM users
2025-05-19 20:36:56,593 INFO sqlalchemy.engine.Engine [cached since 391.7s ago] ()
2025-05-19 20:36:56,599 INFO sqlalchemy.engine.Engine ROLLBACK
(1, 'María López', 'maria@example.com')
(2, 'Pedro Gómez', 'pedro@example.com')


✅ Función genérica para actualizar usuario:

In [44]:
from sqlalchemy import update

In [45]:
def actualizar_usuario_por(filtros, nuevos_datos):
    """
    filtros: dict para ubicar el usuario (por id, name, email, etc.)
    nuevos_datos: dict con las columnas a actualizar
    """
    with engine.connect() as conn:
        condiciones = [getattr(users.c, campo) == valor for campo, valor in filtros.items()]
        stmt = update(users).where(*condiciones).values(**nuevos_datos)
        result = conn.execute(stmt)
        conn.commit()
        return result.rowcount

In [46]:
actualizar_usuario_por({"name":"María López"}, {"name":"Maria Belen López"})

2025-05-19 20:38:04,542 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-19 20:38:04,546 INFO sqlalchemy.engine.Engine UPDATE users SET name=? WHERE users.name = ?
2025-05-19 20:38:04,547 INFO sqlalchemy.engine.Engine [generated in 0.00517s] ('Maria Belen López', 'María López')
2025-05-19 20:38:04,552 INFO sqlalchemy.engine.Engine COMMIT


1

In [47]:
usuarios = leer_todos_los_usuarios()
for usuario in usuarios:
    print(usuario)

2025-05-19 20:38:38,014 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-19 20:38:38,015 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.email 
FROM users
2025-05-19 20:38:38,020 INFO sqlalchemy.engine.Engine [cached since 493.1s ago] ()
2025-05-19 20:38:38,024 INFO sqlalchemy.engine.Engine ROLLBACK
(1, 'Maria Belen López', 'maria@example.com')
(2, 'Pedro Gómez', 'pedro@example.com')


Metodo Patch

la función de update() en SQLAlchemy es "parcial" por naturaleza, como un PATCH: solo actualiza los campos que vos le pases, sin tocar los demás.

- ✅ ¿Qué significa esto?

Cuando hacés:

```python
actualizar_usuario_por({'email': 'a@b.com'}, {'name': 'Nuevo Nombre'})
```

Solo se actualiza la columna name. Si el usuario tiene otros campos como email o id, esos no se modifican.

- 🔄 Entonces, para comportarse como PATCH:

No necesitás nada especial. Solo asegurate de pasar solo los campos que querés actualizar.

- 🛑 Validación (opcional)

Si querés que no actualice si el usuario no existe, podés hacer una verificación previa:

```python
def actualizar_usuario_por(filtros, nuevos_datos):
    with engine.connect() as conn:
        condiciones = [getattr(users.c, campo) == valor for campo, valor in filtros.items()]
        # Verificar si existe
        select_stmt = select(users).where(*condiciones)
        result = conn.execute(select_stmt).mappings().first()
        if not result:
            return 0  # o lanzar una excepción
        # Actualizar solo campos indicados
        update_stmt = update(users).where(*condiciones).values(**nuevos_datos)
        result = conn.execute(update_stmt)
        conn.commit()
        return result.rowcount
```
